## Parameter Efficient Fine Tuning (LoRA) and Reinforcement Learning with human Feedback

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install -U datasets

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch \
    torchdata --quiet
# all from huggingface
%pip install \
    transformers \
    evaluate \
    rouge_score \
    loralib \
    peft --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.9 MB/s eta 0:00:00


In [3]:
%pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.1 MB/s eta 0:00:00


In [4]:
import os
#os.environ["SM_FRAMEWORK"] = "tf.keras"
from datasets import load_dataset
from transformers import (AutoModelForSeq2SeqLM,
                          AutoTokenizer,
                          GenerationConfig,
                          TrainingArguments,
                          Trainer,
                          pipeline,
                          AutoModelForSequenceClassification)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
# trl: Transformer Reinforcement Learning library
# for PPO
# AutoModelForSeq2SeqLMWithValueHead required for PPO
# LengthSampler sample from text various lengths at different points in time number of characters
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [5]:
# load dataset and LLM
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [6]:
# auxiliary code to count number of trainable parameters in a model
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [8]:
print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [ ]:
# perform zero shot inference to test model summarization on dialog dataset
index = 250

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
        model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: There's a car waiting for you just outside the door. Right this way, please.
#Person2#: OK!
#Person1#: Let me put your cases into the trunk and please get in the back.
#Person2#: Thanks!
#Person1#: How was your flight?
#Person2#: It's comfortable, but now I'm a little tired.
#Person1#: We'll reach the Beijing hotel in another ten minutes. When we arrived there, you can go up and have a rest. The hotel has very good service, and it's considered as one of the best hotels here.
#Person2#: Thank you! I lived there when I came to Beijing last time. It's comfortable and beautiful.
#Person1#: If it's convenient for you, Mr. Wu would like to invite you to the banquet in honor of you in the evening.
#Person2#: Thank you! I will. When and where will the dinner be?
#Person1#: At six o'clock in the International Hotel. We'll pick you u

In [ ]:
# preprocess Datasets
def tokenize_function(sample):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in sample["dialogue"]]
    sample['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    sample['labels'] = tokenizer(sample["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return sample

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

## PEFT
Use the LoRA Reparameterization method to fine tune the Flan-T5 model with summarization instructions

In [8]:
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    # target_modules which determine where the smaller matrices are inserted
    # here we target the query and values matrcies in attention blocks
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [10]:
# add lora adapter layers to original llm to be trained
peft_model = get_peft_model(model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


In [ ]:
output_dir = f'drive/MyDrive/peft_rlhf/peft_dialogue_summary_train-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=20,
    logging_steps=1,
    max_steps=100
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

In [ ]:
# train and save model
peft_model_path="./drive/MyDrive/peft_rlhf/peft-dialogue-summary-train-epoch-20"

peft_trainer.train()
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

In [8]:
# load pretrained model
peft_model = PeftModel.from_pretrained(model,
                                       # path to saved peft model
                                       './drive/MyDrive/peft_rlhf/peft-dialogue-summary-checkpoint/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
print(device)

cuda


In [11]:
peft_model.to(device)

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
            

## Evaluate model using ROUGE metrics

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to(device)

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = model.generate(input_ids=input_ids, max_new_tokens=200)
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, max_new_tokens=200)
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])

In [ ]:
df

,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1# asks Ms. Dawson to take a dictation ...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,#Person2# got stuck in traffic and #Person1# s...,#Person2# got stuck in traffic and #Person1# s...
4,#Person2# decides to follow #Person1#'s sugges...,#Person2# got stuck in traffic and #Person1# s...,#Person2# got stuck in traffic and #Person1# s...
5,#Person2# complains to #Person1# about the tra...,#Person2# got stuck in traffic and #Person1# s...,#Person2# got stuck in traffic and #Person1# s...
6,#Person1# tells Kate that Masha and Hero get d...,Kate tells #Person2# Masha and Hero are gettin...,Kate tells #Person2# Masha and Hero are gettin...
7,#Person1# tells Kate that Masha and Hero are g...,Kate tells #Person2# Masha and Hero are gettin...,Kate tells #Person2# Masha and Hero are gettin...
8,#Person1# and Kate talk about the divorce betw...,Kate tells #Person2# Masha and Hero are gettin...,Kate tells #Person2# Masha and Hero are gettin...
9,#Person1# and Brian are at the birthday party ...,Brian remembers his birthday and invites #Pers...,Brian remembers his birthday and invites #Pers...


## compute Rouge Score

In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [ ]:
print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.3725351062275605, 'rouge2': 0.12138811933618107, 'rougeL': 0.27620639623170606, 'rougeLsum': 0.2758134870822362}
PEFT MODEL:
{'rouge1': 0.3725351062275605, 'rouge2': 0.12138811933618107, 'rougeL': 0.27620639623170606, 'rougeLsum': 0.2758134870822362}


## Evaluate on full dataset

In [ ]:
results = pd.read_csv('./drive/MyDrive/peft_rlhf/data/results.csv')
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
peft_model_summaries = results['peft_model_summaries'].values

In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

In [ ]:
print('ORIGINAL MODEL Rouge Score:')
print(original_model_results)
print('PEFT MODEL Rouge Score:')
print(peft_model_results)

ORIGINAL MODEL Rouge Score:
{'rouge1': 0.2334158581572823, 'rouge2': 0.07603964187010573, 'rougeL': 0.20145520923859048, 'rougeLsum': 0.20145899339006135}
PEFT MODEL Rouge Score:
{'rouge1': 0.40810631575616746, 'rouge2': 0.1633255794568712, 'rougeL': 0.32507074586565354, 'rougeLsum': 0.3248950182867091}


In [ ]:
print("Percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.1f}%')

Percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 17.5%
rouge2: 8.7%
rougeL: 12.4%
rougeLsum: 12.3%


## Reinforcment Learning with Human Feedback

Finetune the peft_model to generate less toxic content using META's hate speech model as the reward model.

In [9]:
# load save peft_model and set up for fine-tuning
peft_model = PeftModel.from_pretrained(model,
                                       # path to saved peft model
                                       './drive/MyDrive/peft_rlhf/peft-dialogue-summary-checkpoint/',
                                       torch_dtype=torch.bfloat16,
                                       lora_config=lora_config,
                                       device_map="auto",
                                       is_trainable=True)

In [10]:
def tokenize(sample):
        # Wrap each dialogue with the instruction.
        prompt = f"""
Summarize the following conversation.

{sample["dialogue"]}

Summary:
"""
        sample["input_ids"] = tokenizer.encode(prompt)

        # This must be called "query", which is a requirement of our PPO library.
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

In [11]:
# filter dataset
def filter_dataset(model_name,
                  dataset_name,
                  input_min_text_length,
                  input_max_text_length, tokenize=tokenize):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model_name (str): Tokenizer model name.
    - dataset_name (str): Name of the dataset to load.
    - input_min_text_length (int): Minimum length of the dialogues.
    - input_max_text_length (int): Maximum length of the dialogues.

    Returns:
    - dataset_splits (datasets.dataset_dict.DatasetDict): Preprocessed dataset containing train and test parts.
    """

    # use only the train set
    dataset = load_dataset(dataset_name, split="train")

    # Filter the dialogues of length between input_min_text_length and input_max_text_length characters.
    dataset = dataset.filter(lambda x: len(x["dialogue"]) > input_min_text_length and len(x["dialogue"]) <= input_max_text_length, batched=False)


    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

    # Tokenize each dialogue.
    dataset = dataset.map(tokenize, batched=False)
    dataset.set_format(type="torch")

    # Split the dataset into train and test parts.
    dataset_splits = dataset.train_test_split(test_size=0.2, shuffle=False, seed=42)

    return dataset_splits

dataset = filter_dataset(model_name=model_name,
                        dataset_name=huggingface_dataset_name,
                        input_min_text_length=200,
                        input_max_text_length=1000)


Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/10022 [00:00<?, ? examples/s]

In [12]:
# PPO model - this will be used to fine tune the LLM with RL
# PPO will be used to optimize the RL policy against the reward model
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True)

In [13]:
print(print_number_of_trainable_model_parameters(ppo_model))


trainable model parameters: 3539713
all model parameters: 251117569
percentage of trainable model parameters: 1.41%


In [14]:
# during PPO only the parameters associated with the value head is updated.
print(ppo_model.v_head)

ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [15]:
# create ref model - this is not going to be updated
# reference model for RLHF when used with KL divergence
# this ensure that while the reward is being optimize it keeps relevance
ref_model = create_reference_model(ppo_model)


## Reward Model


In [16]:
# Use META AI's RoBERTa-based hate speech model
# This model will output logits and then predict probabilities across two classes:
# 'nothate' and 'hate'. The logits of the output 'nothate' will be taken as a positive reward.
# while 'hate' logits will have a low reward
# Then, the model will be fine-tuned with PPO using those reward values.
reward_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
reward_tokenizer = AutoTokenizer.from_pretrained(reward_model_name, device_map="auto")
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_name, device_map="auto")


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [17]:
# 0 corresponds to 'nothate'
print(reward_model.config.id2label)

{0: 'nothate', 1: 'hate'}


In [18]:
# set up reward model inference pipeline
reward_model_pipe = pipeline("sentiment-analysis",
                          model=reward_model_name,
                          device=device)
reward_logits_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # 'none' to retrieve raw logits.
    "batch_size": 16
}


toxic_eval = evaluate.load("toxicity", reward_model_name,
                           module_type="measurement",
                           toxic_label="hate")



In [38]:
# compute toxicity of dialouges
def evaluate_toxicity(model,
                      toxicity_evaluator,
                      tokenizer,
                      dataset,
                      num_samples):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model (trl model): Model to be evaluated.
    - toxicity_evaluator (evaluate_modules toxicity metrics): Toxicity evaluator.
    - tokenizer (transformers tokenizer): Tokenizer to be used.
    - dataset (dataset): Input dataset for the evaluation.
    - num_samples (int): Maximum number of samples for the evaluation.

    Returns:
    tuple: A tuple containing two numpy.float64 values:
    - mean (numpy.float64): Mean of the samples toxicity.
    - std (numpy.float64): Standard deviation of the samples toxicity.
    """

    max_new_tokens=100
    toxicities = []
    input_texts = []
    for i, sample in tqdm(enumerate(dataset)):
        input_text = sample["query"]

        if i > num_samples:
            break

        input_ids = tokenizer(input_text, return_tensors="pt",
                              padding=True).input_ids
        input_ids = input_ids.to("cuda")

        generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                             top_k=0.0,
                                             top_p=1.0,
                                             do_sample=True)

        response_token_ids = model.generate(input_ids=input_ids,
                                            generation_config=generation_config)

        generated_text = tokenizer.decode(response_token_ids[0],
                                          skip_special_tokens=True)

        toxicity_score = toxic_eval.compute(predictions=[(input_text + " " +
                                                          generated_text)])

        toxicities.extend(toxicity_score["toxicity"])

    # Compute mean & std using np.
    mean = np.mean(toxicities)
    std = np.std(toxicities)

    return mean, std

In [63]:
# model toxicity before finetuning
mean, std = evaluate_toxicity(model=ref_model,
                             toxicity_evaluator=toxic_eval,
                             tokenizer=tokenizer,
                             dataset=dataset["test"],
                             num_samples=15)


16it [00:32,  2.02s/it]


In [60]:
print(f"Toxicity [mean, std] before detox: [{mean, std}]")

Toxicity [mean, std] before detox: [(0.0297900027810381, 0.029646099338437457)]


## PPO

In [43]:
# needed for PPO
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

learning_rate=1.41e-5
max_ppo_epochs=10
mini_batch_size=4
batch_size=16

config = PPOConfig(
    model_name=model_name,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,
                         ref_model=ref_model,
                         tokenizer=tokenizer,
                         dataset=dataset["train"],
                         data_collator=collator)

In [44]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

max_ppo_steps = 10
for epoch in tqdm(range(ppo_trainer.config.ppo_epochs), "epoch: "):
  for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if step >= max_ppo_steps:
        break

    prompt_tensors = batch["input_ids"]

    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()

        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)

        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # This needs to be called "response"
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]

    # Compute reward outputs.
    query_response_pairs = [q + r for q, r in zip(batch["query"],
                                                  batch["response"])]
    rewards = reward_model_pipe(query_response_pairs, **reward_logits_kwargs)

    # You use the 'nothate' index because
    # the score is positive for 'nothate' class
    reward_tensors = [torch.tensor(reward[0]["score"]) for reward in rewards]

    # Run PPO step.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))


epoch:   0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:40, 40.90s/it]

objective/kl: 18.92055892944336
ppo/returns/mean: -0.13236938416957855
ppo/policy/advantages_mean: 0.022820167243480682
---------------------------------------------------------------------------------------------------



2it [01:15, 37.38s/it]

objective/kl: 21.90687370300293
ppo/returns/mean: -0.35979604721069336
ppo/policy/advantages_mean: 0.004667850211262703
---------------------------------------------------------------------------------------------------



3it [01:46, 34.36s/it]

...

objective/kl: 13.22803020477295
ppo/returns/mean: 0.7262565493583679
ppo/policy/advantages_mean: 0.021634941920638084
---------------------------------------------------------------------------------------------------



9it [04:15, 28.17s/it]

objective/kl: 12.825996398925781
ppo/returns/mean: 0.8254314661026001
ppo/policy/advantages_mean: 0.00528346560895443
---------------------------------------------------------------------------------------------------



10it [04:44, 28.45s/it]
epoch: 100%|██████████| 10/10 [49:32<00:00, 297.26s/it]

objective/kl: 11.241851806640625
ppo/returns/mean: 1.0278315544128418
ppo/policy/advantages_mean: 0.021164540201425552
---------------------------------------------------------------------------------------------------


In [53]:
ppo_model_path="./drive/MyDrive/peft_rlhf/"

ppo_trainer.model.save_pretrained(ppo_model_path)


In [65]:
mean_after_detox, std_after_detox = evaluate_toxicity(model=ppo_model,
                                                      toxicity_evaluator=toxic_eval,
                                                      tokenizer=tokenizer,
                                                      dataset=dataset["test"],
                                                      num_samples=15)
print(f'toxicity [mean, std] after detox: [{mean_after_detox}, {std_after_detox}]')

16it [00:20,  1.31s/it]

toxicity [mean, std] after detox: [0.012295305787120014, 0.009127263851730166]


In [ ]:
# print models parameters and device
#for i in ppo_model.named_parameters():
#    print(f"{i[0]} -> {i[1].device}")

In [66]:
mean_improvement = (mean - mean_after_detox) / mean
std_improvement = (std - std_after_detox) / std

print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

Percentage improvement of toxicity score after detoxification:
mean: 53.88%
std: 77.93%
